In [ ]:
#gpu정보 확인
!nvidia-smi
# watch nvidia-smi 커널에서 계속 확인
# nvidia-smi -lms 실시간 확인
# CUDA_VISIBLE_DEVICES=0 python3 file.py / GPU 0번만 사용

In [1]:
import tensorflow as tf

tf.reset_default_graph()

In [2]:
def parser(serialized_example):
    features = {
        'age': tf.FixedLenFeature([1], tf.int64),
        'img': tf.FixedLenFeature([61*49], tf.int64)
    }
    
    parsed_feature = tf.parse_single_example(serialized_example, features)
    age = parsed_feature['age']
    img = parsed_feature['img']
    return age, img

In [3]:
train_data_dir = './cnn_dataset/face_train.tfrecord'
test_data_dir = './cnn_dataset/face_test.tfrecord'

train_dataset = tf.contrib.data.TFRecordDataset(train_data_dir).map(parser)
train_dataset = train_dataset.shuffle(7777).batch(32)
#train_dataset = train_dataset.repeat(10)
itr = train_dataset.make_one_shot_iterator()

age, img = itr.get_next()

"""
이미지 잘들어오는지 확인
with tf.Session() as sess:
    _age, _img = sess.run([age, img])
    print(_age)
    print(_img)
"""

img = tf.reshape(img, [-1, 61, 49, 1])
# int -> float (int 산술연산 잘 안되기 때문에 float으로 바꿔준다.)
img = tf.cast(img, tf.float32)


age = tf.reshape(age, [-1])
age = tf.one_hot(age, depth=3, axis=-1, dtype = tf.float32)

"""
이미지 형태 확인
with tf.Session() as sess:
    _age, _img = sess.run([age, img])
    print(_age)
    print(_img)
"""

activation = tf.nn.relu
reuse = tf.AUTO_REUSE
#kernel_size 3 or [3, 3] 가로 세로 3 * 3
conv1 = tf.layers.conv2d(img, filters=16, kernel_size=3, padding='SAME', 
                         activation=activation, reuse=reuse, name='conv1')
print('conv1 : ', conv1.shape)
conv2 = tf.layers.conv2d(conv1, filters=32, kernel_size=3, padding='SAME',
                        activation=activation, reuse=reuse, name='conv2')
print('conv2 : ', conv2.shape)
#pool_size 얼마나 줄일꺼냐, strides 몇칸씩 이동?
pool2 = tf.layers.max_pooling2d(conv2, pool_size=2, strides=2)
print('pool2 : ', pool2.shape)

conv3 = tf.layers.conv2d(pool2, filters=64, kernel_size=3, padding='SAME',
                        activation=activation, reuse=reuse, name='conv3')
print('conv3 : ', conv3.shape)

pool3 = tf.layers.max_pooling2d(conv3, pool_size=2, strides=2)
print('pool3 : ', pool3.shape)

flat = tf.layers.flatten(pool3)
print('flat : ', flat.shape)

fc1 = tf.layers.dense(flat, 5000, activation=tf.nn.relu, reuse=reuse, name='fc1')
print('fc1 : ', fc1.shape)
fc2 = tf.layers.dense(fc1, 1000, activation=tf.nn.relu, reuse=reuse, name='fc2')
print('fc2 : ', fc2.shape)
out = tf.layers.dense(fc2, units=3, reuse=reuse, name='out')
print('out : ', out.shape)


loss = tf.losses.softmax_cross_entropy(age, out)
train_op = tf.train.GradientDescentOptimizer(1e-5).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):
        _, _loss = sess.run([train_op, loss])
        print('step: {}, loss: {}'.format(i, _loss))

Instructions for updating:
Use `tf.data.TFRecordDataset`.
conv1 :  (?, 61, 49, 16)
conv2 :  (?, 61, 49, 32)
pool2 :  (?, 30, 24, 32)
conv3 :  (?, 30, 24, 64)
pool3 :  (?, 15, 12, 64)
flat :  (?, 11520)
fc1 :  (?, 5000)
fc2 :  (?, 1000)
out :  (?, 3)
step: 0, loss: 34.97539138793945
step: 1, loss: 10.511804580688477
step: 2, loss: 2.8064095973968506
step: 3, loss: 5.196539878845215
step: 4, loss: 2.4722490310668945
step: 5, loss: 2.581576108932495
step: 6, loss: 2.278045177459717
step: 7, loss: 2.3023054599761963
step: 8, loss: 3.5663747787475586
step: 9, loss: 2.8424630165100098
